In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

In [2]:
dbpedia_df = pd.read_csv('../data/train.csv', skiprows=1, names = ['Label', 'Name', 'Text'])

### DBPedia classes

- Company
- EducationalInstitution
- Artist
- Athlete
- OfficeHolder
- MeanOfTransportation
- Building
- NaturalPlace
- Village
- Animal
- Plant
- Album
- Film
- WrittenWork

In [4]:
dbpedia_df.shape

(559999, 3)

In [5]:
dbpedia_df['Label'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int64)

In [6]:
dbpedia_df = dbpedia_df.sample(10000, replace=False)
dbpedia_df.sample(10)

Label                                  Name  \
130396      4                          Isabel Cueto   
205404      6                  USS Transfer (IX-46)   
90000       3                        Vadim Delaunay   
363252     10                             Loxostege   
360482     10                             Paraptica   
70393       2   North Florida Christian High School   
377451     10                       Armenoceratidae   
65282       2  Camden High School (Camden New York)   
382136     10                             Antapamea   
112975      3                       Nachman Fahrner   

                                                     Text  
130396   Isabel Cueto (born 3 December 1968 in Kehl) i...  
205404   USS Transfer was a steam-propelled derrick-ri...  
90000    Vadim Nikolaevich Delaunay (Russian: Вади́м Н...  
363252   Loxostege is a genus of moths of the Crambida...  
360482   Paraptica are a genus of insect belonging to ...  
70393    North Florida Christian School is a private s...  
377451   Armenoceratidae is a family of early Paleozoi...  
65282    Camden High School is a public high school in...  
382136   Antapamea is a genus of moths of the Noctuida...  
112975   Nachman Fahrner (born September 21 1972) is a...

In [7]:
X = dbpedia_df['Text']
Y = dbpedia_df['Label']

In [8]:
X.head()

158174     Brahima Korbeogo (born 23 January 1975) is a ...
310301     The Mâloasa River is a tributary of the Sebeş...
141049     Stephen Reynolds (born 11 June 1992 in Rothes...
397155     Hyposmocoma corvina is a species of moth of t...
192420     Jorge Jiménez Cantú (1914–2005) was a Mexican...
Name: Text, dtype: object

In [9]:
Y.head()

158174     4
310301     8
141049     4
397155    10
192420     5
Name: Label, dtype: int64

In [10]:
def summarize_classification(y_test, y_pred):

    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)
    prec = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    print("Length of testing data: ", len(y_test))
    print("accuracy_count : " , num_acc)
    print("accuracy_score : " , acc)
    print("precision_score : " , prec)
    print("recall_score : ", recall)

### Count Vectorizer

In [11]:
count_vectorizer = CountVectorizer()
feature_vector = count_vectorizer.fit_transform(X)
feature_vector.shape

(10000, 48296)

In [13]:
X_dense = feature_vector.todense()

In [14]:
X_dense.shape

(10000, 48296)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X_dense, Y, test_size = 0.2)

In [16]:
x_train.shape, x_test.shape

((8000, 48296), (2000, 48296))

In [17]:
y_train.shape, y_test.shape

((8000,), (2000,))

In [18]:
clf = GaussianNB().fit(x_train, y_train)

In [19]:
y_pred = clf.predict(x_test)
summarize_classification(y_test, y_pred)

Length of testing data:  2000
accuracy_count :  1494
accuracy_score :  0.747
precision_score :  0.7564730348049717
recall_score :  0.747


### Count Vector with Stop word removal

In [21]:
count_vectorizer = CountVectorizer(stop_words='english')
feature_vector = count_vectorizer.fit_transform(X)
feature_vector.shape

(10000, 48011)

In [22]:
X_dense = feature_vector.todense()
X_dense.shape

(10000, 48011)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X_dense, Y, test_size = 0.2)

In [24]:
clf = GaussianNB().fit(x_train, y_train)
y_pred = clf.predict(x_test)
summarize_classification(y_test, y_pred)

Length of testing data:  2000
accuracy_count :  1443
accuracy_score :  0.7215
precision_score :  0.7352948570185506
recall_score :  0.7215


### Count Vector with Stop word and Frequency Filtering

In [25]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [26]:
tokens = word_tokenize("\n".join(X.values))
len(tokens)

504816

In [28]:
freq = FreqDist(tokens)
freq

FreqDist({'.': 23490, 'the': 23439, 'in': 16072, 'of': 15669, 'is': 13581, 'a': 13173, 'and': 12857, '(': 7165, ')': 7161, 'was': 6786, ...})

In [29]:
frequent_words = []

for key, value in freq.items():
    if value >= 100:
        frequent_words.append(key.lower())
        
len(frequent_words)

492

In [30]:
frequent_words[:25]

['(',
 'born',
 '23',
 'january',
 ')',
 'is',
 'a',
 'footballer',
 'who',
 'currently',
 'plays',
 'for',
 '.',
 'the',
 'river',
 'tributary',
 'of',
 'the',
 'in',
 '11',
 'june',
 '1992',
 'as',
 'he',
 'st.']

In [32]:
from sklearn.feature_extraction import text 
stop_words = text.ENGLISH_STOP_WORDS.union(frequent_words)

In [33]:
count_vectorizer = CountVectorizer(stop_words=stop_words)
feature_vector = count_vectorizer.fit_transform(X)
feature_vector.shape

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['st'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(10000, 47679)

In [34]:
X_dense = feature_vector.todense()

In [35]:
x_train, x_test, y_train, y_test = train_test_split(X_dense, Y, test_size = 0.2)

In [36]:
clf = GaussianNB().fit(x_train, y_train)
y_pred = clf.predict(x_test)
summarize_classification(y_test, y_pred)

Length of testing data:  2000
accuracy_count :  1350
accuracy_score :  0.675
precision_score :  0.681238460843105
recall_score :  0.675


### Count Vector with BiGrams

In [37]:
count_vectorizer = CountVectorizer(ngram_range=(2, 2))
feature_vector = count_vectorizer.fit_transform(X)
feature_vector.shape

(10000, 216192)

In [39]:
X_dense = feature_vector.todense()

MemoryError: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_dense, Y, test_size = 0.2)

In [ ]:
clf = GaussianNB().fit(x_train, y_train)
y_pred = clf.predict(x_test)
summarize_classification(y_test, y_pred)

Length of testing data:  2000
accuracy_count :  1812
accuracy_score :  0.906
precision_score :  0.9071432275572098
recall_score :  0.906